In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'C:\Users\mebratie\Desktop\KR\KR_Regression\Data\Henon_non_chaotic_T10_20_25_30_35_40_45_50\Henon_non_chaotic_T10_35'
def generate_random_values():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_random_values_based_on_c():
    a1 = -0.5
    a2 = 0.5
    a3 = -0.1
    a4 = 0.1
    x1 = np.random.uniform(a1, a2)
    x2 = np.random.uniform(a1, a2)
    x3 = np.random.uniform(a3, a4)
    x4 = np.random.uniform(a3, a4)
    return x1, x2, x3, x4
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([x3, x4, -x1 - 2*x1*x2, -x2 - x1**2 + x2**2])
        normalized_f = normalize(f)
        return normalized_f
    def compute_energy(x1, x2, x3, x4):
        return 0.5 * (x3**2 + x4**2) + 0.5 * (x1**2 + x2**2) + x1**2 * x2 - (1/3) * x2**3
    num_trajectories = 10
    t = np.linspace(0, 10, 35) # 40 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
        final_state = sol[-1, :]
        E = compute_energy(*final_state)
        initial_conditions_to_print.append((initial_condition, E))
    num_variables = 4 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    for r, (initial_conditions, E) in enumerate(initial_conditions_to_print):
        print(f"Energy for Trajectory {r+1}: {E}")
    output_directory1 = r'C:\Users\mebratie\Desktop\KR\KR_Regression\results\Henon_non_chaotic_T10_20_25_30_35_40_45_50\Henon_non_chaotic_T10_35'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('C:\\Users\\mebratie\\Desktop\\KR\\KR_Regression\\Data\\Henon_non_chaotic_T10_20_25_30_35_40_45_50\\Henon_non_chaotic_T10_35\\trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(10)]  # number of trajectories
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 11):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4):
(0.19243515339970585, 0.18390172008794392, 0.05943992074937429, -0.045228432522012654)
(-0.009415096374007725, 0.3099819709633067, -0.024275499712063858, -0.022209869469596105)
(0.22511338932164304, 0.17774491209595866, -0.009663182670330728, -0.0070085098218611586)
(0.29031952191396093, -0.06911010110765137, 0.06926851237337447, -0.03996905287906449)
(-0.05714199305273515, -0.14093243033052316, -0.07566122681700527, 0.04516247875989127)
(0.1201567997425168, 0.0007124156503098833, -0.08268652890870075, 0.053333284980919826)
(-0.1544682075979752, 0.024157033445889264, -0.011533505480503886, 0.05244455165919079)
(-0.22068290496180476, -0.19454029892008007, -0.06444442028461211, 0.03461104274642263)
(-0.3504642581481724, -0.05968925793861879, 0.04105037571732342, 0.09190403316966486)
(-0.345916920440409, 0.0714603713956089, 0.08216819520761595, 0.08818270154998784)
Energy for Trajectory 1: 0.04295188868838274
Energy for Trajectory 2: 0.038728882231702684
Ene